# Kaleidocycle Exploration


In [ ]:
%matplotlib widget
import sys
import os

# Add parent src directory to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from ipywidgets import Dropdown, FloatSlider, interact, Checkbox
from scipy.optimize import minimize

# Import kaleidocycle functions
from kaleidocycle import (
    Kaleidocycle,
    ConstraintConfig,
    SolverOptions,
    bending_energy,
    binormals_to_tangents,
    tangents_to_curve,
    compute_tetrahedron_vertices,
    constraint_penalty,
    dipole_energy,
    compute_axis,
    optimize_cycle,
    random_hinges,
    torsion_energy,
    pairwise_curvature,
    pairwise_cosines,
    curvature_recursion,
    curvature_recursion_from_tangents,
    cos_invariant,
    # Visualization functions
    create_rotation_animation,
    plot_band,
    plot_curve,
    plot_energy_comparison,
    plot_vertex_values,
    plot_hinges,
    mean_cosine,
    paper_model,
    # Import/Export functions
    export_json,
    export_csv,
    import_json,
    import_csv,
    format_report,
    # Transformations
    involution,
    mirror,
    negative_twist,
    reverse,
    subdivide,
    transform_kaleidocycle,
)
from kaleidocycle import (
    optimize_with_linking_constraint,
    compute_linking_number,
    ConstraintConfig,
    random_hinges,
)

print("✓ All imports successful")
print("✓ Interactive plots enabled (if ipympl is installed)")

## Find Kaleidocycles via Optimization

Conjecture: When non-oriented or odd n, the minimisers of bending attain the extremum of the torsion. 

In [ ]:
# Set parameters for optimization
# note: objective 'mean_cos' is meaningless with oriented n:even or non-oriented n:odd (always -1.0)
#       objective 'neg_mean_cos' is meaningless with oriented (always 1.0)

n=8
oriented=False
objective = 'neg_mean_cos' # 'bending' #
seed=19
#method='BFGS'
maxiter=15000

initial = random_hinges(n, seed=seed, oriented=oriented).as_array()
config = ConstraintConfig(oriented=oriented, enforce_anchors=False, constant_torsion=True)

# Use the built-in optimizer to minimize the objective function
result = optimize_cycle(
    initial,
    config,
    objective=objective,
    options=SolverOptions(maxiter=maxiter, penalty_weight=10000.0, use_constraint_solver=True),
)

print(f"scipy success flag: {result.success}")
if not result.success:
    print(f"  message: {result.scipy_result.message}")
print(f"Objective value: {result.scipy_result.fun:.6f}")

print("\n\n" + format_report(result.hinges, config))

In [ ]:
# create Kaleidocycle object from optimized hinges
kc = Kaleidocycle(hinges=result.hinges)
plot_vertex_values(kc.curvatures)

In [ ]:
kc.plot()

In [ ]:
print(curvature_recursion(kc.curvatures, oriented=oriented))
print(cos_invariant(kc.curvatures, oriented=oriented))

In [ ]:
# Transformations example
kc2 = transform_kaleidocycle(kc, "subdivide", divisions=3)
kc2.plot()
print(kc2.report())


## Export and Import Kaleidocycle Configurations

Save and load kaleidocycle configurations in JSON (full data) or CSV (hinges only) formats.

In [ ]:
# Example: Export and import an optimized kaleidocycle
print(f"Optimized n={n} kaleidocycle")
oriented_str = 'oriented' if oriented else 'nonoriented'
filename = f'kaleidocycle_k{n}_{oriented_str}_{objective}'

# Export to JSON (all data)
export_json(kc,filename+'.json')
print(f"\n✓ Exported to {filename}.json")

# Export to CSV (hinges only)
export_csv(
    result.hinges,
    filename+'.csv',
    header=True,
)
print(f"✓ Exported to {filename}.csv includes hinges only (n+1 rows × 3 columns)")


In [ ]:
# Import from JSON
kc = import_json(filename+'.json')
print(kc.report())

# Import from CSV
loaded_csv = import_csv(filename+'.csv')

print(f"\nLoaded from CSV:")
print(f"  hinges shape: {loaded_csv.shape}")
print(f"  n = {loaded_csv.shape[0] - 1}")

# Verify they match
if np.allclose(kc.hinges, loaded_csv):
    print("\n✓ JSON and CSV hinges match!")
else:
    print("\n✗ Warning: JSON and CSV hinges don't match")


## Hinge manipulation (without using Kaleidocycle class)

In [ ]:
binormals = result.hinges
# Compute mid-axes (tangent vectors)
tangents = binormals_to_tangents(binormals)
print(f"Tangent vectors: {tangents.shape}")

# Compute curve (accumulated positions)
curve = tangents_to_curve(tangents)
print(f"curve points: {curve.shape}")
print(f"first and last curve difference (should be close to zero): {np.linalg.norm(curve[-1]-curve[0])}")

# Compute curvature
curvature = pairwise_curvature(binormals, signed=True)
print(f"\nCurvature values (radians):")
print(curvature)
print(f"Curvature (degrees): {np.degrees(curvature)}")
print(f"Mean curvature: {np.degrees(np.mean(curvature)):.2f}°")

# Compute torsion
torsion = pairwise_cosines(binormals)
print(f"\nTorsion values (cosine):")
print(f"Mean torsion: {(np.mean(torsion)):.6f}")
print(f"Std torsion: {(np.std(torsion)):.6f}")


In [ ]:
# Compute and visualize the optimized curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6), subplot_kw={'projection': '3d'})

plot_curve(curve, ax=ax1, title='curve', color='forestgreen')

# Plot band structure
plot_band(
    curve,
    binormals,
    ax=ax2,
    width=0.15,
    title='Optimized Band',
    facecolor='lightcoral',
    edgecolor='darkred',
    alpha=0.7,
)

plt.tight_layout()
plt.show()

In [ ]:
# Use subset of binormals and curvature
axis = compute_axis(binormals[:4], curvature[:3])

print("Rotation Axis:")
print(f"  A = [{axis[0]:.4f}, {axis[1]:.4f}, {axis[2]:.4f}]")
print(f"  |A| = {np.linalg.norm(axis):.4f}")
print(f"\nNormalized: [{axis[0]/np.linalg.norm(axis):.4f}, "
        f"{axis[1]/np.linalg.norm(axis):.4f}, "
        f"{axis[2]/np.linalg.norm(axis):.4f}]")

# Verify: A · B[i] should equal tan(K[i]/2)
print("\nVerification (A · B[i] = tan(K[i]/2)):")
for i in range(len(binormals)-1):
    lhs = np.dot(axis, binormals[i])
    rhs = np.tan(curvature[i] / 2)
    print(f"  i={i}: {lhs:.6f} ≈ {rhs:.6f} (diff={abs(lhs-rhs):.2e})")


## Constrained Optimization with Lk

Conjecture: Local minima share similar properties to Mobius kaleidocycles.

In [ ]:
oriented =  True

config = ConstraintConfig(oriented=oriented, constant_torsion=True, enforce_anchors=False)
initial_binormals = random_hinges(9, seed=40, oriented=oriented).as_array()
result = optimize_with_linking_constraint(
    initial_binormals,
    target_linking=4,
    config=config,
    objective="bending",
)

final_lk = compute_linking_number(result.hinges)
print(f"Achieved: {final_lk:.3f}π, Energy: {result.energy:.3f}")
print("\n\n" + format_report(result.hinges, config))

In [ ]:
# plot result
kc = Kaleidocycle(hinges=result.hinges)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6), subplot_kw={'projection': '3d'})

plot_curve(kc.curve, ax=ax1)
kc.plot(ax=ax2)

plt.tight_layout()
plt.show()

In [ ]:
export_json(kc, 'kaleidocycle_k9_oriented_lk4_bending.json')
print(f"\n✓ Exported to kaleidocycle_k9_oriented_lk4_bending.json")

## Generate paper model

In [ ]:

ax = paper_model(
    result.hinges,
    facecolors=["lightblue", "lightcoral", "lightgreen"],
    edgecolor="black",
    linewidth=2.0,
    alpha=0.5,
)

plt.savefig("paper_template.png")
plt.show()